In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import sklearn as skl
from sklearn.decomposition import PCA

#  Import and read the charity_data.csv.
import pandas as pd 
CleanedDF = pd.read_csv("Resources/CleanedDF.csv")
CleanedDF = CleanedDF.drop(['Unnamed: 0', 'SPECIAL_CONSIDERATIONS', 'STATUS'], axis=1)
CleanedDF

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [2]:
CleanedDF.nunique()

APPLICATION_TYPE       9
AFFILIATION            6
CLASSIFICATION         6
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [3]:
AppType = CleanedDF['APPLICATION_TYPE'].value_counts()
AppType

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
appReplace = list(AppType[AppType<1000].index)

# Replace in dataframe
for app in appReplace:
    CleanedDF['APPLICATION_TYPE'] = CleanedDF['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
CleanedDF['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [5]:
IncAmt = CleanedDF['INCOME_AMT'].value_counts()
IncAmt

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
CleanedDF['INCOME_AMT'] = CleanedDF['INCOME_AMT'].replace(['10000-24999', '1-9999', '25000-99999'], '1-99999')
CleanedDF['INCOME_AMT'] = CleanedDF['INCOME_AMT'].replace(['1M-5M', '5M-10M', '10M-50M', '50M+'], '1M+')
CleanedDF['INCOME_AMT'].value_counts()

0                24388
1-99999           5018
100000-499999     3374
1M+               1519
Name: INCOME_AMT, dtype: int64

In [7]:
CleanedDF.nunique()

APPLICATION_TYPE       6
AFFILIATION            6
CLASSIFICATION         6
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             4
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [8]:
appDummies = pd.get_dummies(CleanedDF)
appDummies

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-99999,INCOME_AMT_100000-499999,INCOME_AMT_1M+
0,5000,1,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0
2,5000,0,0,0,0,0,1,0,1,0,...,0,1,1,0,0,0,1,0,0,0
3,6692,1,0,0,1,0,0,0,1,0,...,1,0,0,0,0,1,0,1,0,0
4,142590,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,1,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
34295,5000,0,0,0,0,1,0,0,1,0,...,0,1,1,0,0,0,1,0,0,0
34296,5000,0,0,0,1,0,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0
34297,5000,1,0,0,0,0,1,0,0,0,...,0,1,1,0,0,0,1,0,0,0


In [9]:
# Split our preprocessed data into our features and target arrays
x=appDummies.drop('IS_SUCCESSFUL', axis=1).values
y=appDummies['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
xTrain, xTest, yTrain, yTest = train_test_split(x, y, random_state=34)

In [10]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(xTrain)

# Scale the data
X_train_scaled = X_scaler.transform(xTrain)
X_test_scaled = X_scaler.transform(xTest)
ScaledtraintwoDF = pd.DataFrame(X_train_scaled)
ScaledtesttwoDF = pd.DataFrame(X_test_scaled)

In [11]:
!pip install keras-tuner

In [12]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=50,
        step=2), activation=activation, input_dim=2))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=50,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [13]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2)

In [14]:
# Run the kerastuner search for best hyperparameters
tuner.search(ScaledtraintwoDF[ScaledtraintwoDF.columns[0:2]],yTrain,epochs=100,validation_data=(ScaledtesttwoDF[ScaledtesttwoDF.columns[0:2]],yTest))

Trial 508 Complete [00h 01m 31s]
val_accuracy: 0.5465888977050781

Best val_accuracy So Far: 0.5465888977050781
Total elapsed time: 01h 27m 36s
INFO:tensorflow:Oracle triggered exit


In [15]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 25,
 'num_layers': 2,
 'units_0': 9,
 'units_1': 27,
 'units_2': 11,
 'units_3': 5,
 'units_4': 43,
 'units_5': 21,
 'tuner/epochs': 100,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(xTrain[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=best_hyper['first_units'], activation=best_hyper['activation'], input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=best_hyper['units_1'], activation=best_hyper['activation']))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=best_hyper['units_2'], activation=best_hyper['activation']))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=best_hyper['units_3'], activation=best_hyper['activation']))

# fifth hidden layer
nn.add(tf.keras.layers.Dense(units=best_hyper['units_4'], activation=best_hyper['activation']))

# sixth hidden layer
nn.add(tf.keras.layers.Dense(units=best_hyper['units_5'], activation=best_hyper['activation']))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 25)                825       
                                                                 
 dense_14 (Dense)            (None, 27)                702       
                                                                 
 dense_15 (Dense)            (None, 11)                308       
                                                                 
 dense_16 (Dense)            (None, 5)                 60        
                                                                 
 dense_17 (Dense)            (None, 43)                258       
                                                                 
 dense_18 (Dense)            (None, 21)                924       
                                                                 
 dense_19 (Dense)            (None, 1)                

In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled, yTrain, epochs=best_hyper['tuner/epochs'])

Epoch 1/100
804/804 [==============================] - 1s 871us/step - loss: 0.5877 - accuracy: 0.7058
Epoch 2/100
804/804 [==============================] - 1s 802us/step - loss: 0.5608 - accuracy: 0.7256
Epoch 3/100
804/804 [==============================] - 1s 762us/step - loss: 0.5578 - accuracy: 0.7268
Epoch 4/100
804/804 [==============================] - 1s 758us/step - loss: 0.5561 - accuracy: 0.7271
Epoch 5/100
804/804 [==============================] - 1s 753us/step - loss: 0.5557 - accuracy: 0.7290
Epoch 6/100
804/804 [==============================] - 1s 789us/step - loss: 0.5549 - accuracy: 0.7278
Epoch 7/100
804/804 [==============================] - 1s 770us/step - loss: 0.5533 - accuracy: 0.7300
Epoch 8/100
804/804 [==============================] - 1s 765us/step - loss: 0.5534 - accuracy: 0.7296
Epoch 9/100
804/804 [==============================] - 1s 778us/step - loss: 0.5529 - accuracy: 0.7304
Epoch 10/100
804/804 [==============================] - 1s 773us/step - l

804/804 [==============================] - 1s 776us/step - loss: 0.5448 - accuracy: 0.7341
Epoch 80/100
804/804 [==============================] - 1s 781us/step - loss: 0.5450 - accuracy: 0.7336
Epoch 81/100
804/804 [==============================] - 1s 804us/step - loss: 0.5449 - accuracy: 0.7345
Epoch 82/100
804/804 [==============================] - 1s 778us/step - loss: 0.5449 - accuracy: 0.7346
Epoch 83/100
804/804 [==============================] - 1s 795us/step - loss: 0.5450 - accuracy: 0.7346
Epoch 84/100
804/804 [==============================] - 1s 768us/step - loss: 0.5453 - accuracy: 0.7341
Epoch 85/100
804/804 [==============================] - 1s 774us/step - loss: 0.5454 - accuracy: 0.7346
Epoch 86/100
804/804 [==============================] - 1s 781us/step - loss: 0.5443 - accuracy: 0.7336
Epoch 87/100
804/804 [==============================] - 1s 776us/step - loss: 0.5446 - accuracy: 0.7339
Epoch 88/100
804/804 [==============================] - 1s 784us/step - loss:

In [24]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,yTest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5545 - accuracy: 0.7327 - 236ms/epoch - 882us/step
Loss: 0.5544573664665222, Accuracy: 0.7327113747596741


In [25]:
# Export our model to HDF5 file
nn.save('h5_files/AlphabetSoupCharity_Optimization.h5')
CleanedDF.to_csv('Resources/optimized.csv')